#####Libraries
***

In [ ]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 39.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 09:49:32--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 09:49:32--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownl

#####Load Dataset
***

In [ ]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,106.605820,81.597748,158.413422,136.604294,136.802216,256.874268,160.932495,203.371292,218.499496,161.456635,...,102.610237,259.269989,252.285995,33.223026,120.749817,398.641052,339.450012,224.728775,169.534698,56.981373
2021-06-02,105.836266,82.125130,156.835663,135.604462,137.594070,257.275818,161.699493,205.369476,215.377518,161.811569,...,102.156342,262.170013,249.654388,33.380478,118.805557,397.788330,338.470001,227.733353,171.020294,57.433754
2021-06-03,106.605820,82.422363,159.030807,135.212387,134.990860,258.382507,159.350494,205.707352,215.249924,162.205963,...,102.345467,257.790009,249.567001,32.563683,117.322823,399.856415,336.119995,226.196365,170.178467,57.659946
2021-06-04,106.748322,82.489479,159.834412,136.084778,138.425507,259.939758,160.311005,204.404175,216.300385,162.679199,...,102.544052,263.040009,251.868423,32.662090,117.313004,397.582550,332.070007,228.209335,170.990555,57.914417
2021-06-07,107.365868,82.144287,159.295410,135.271164,137.554474,259.401062,159.900497,204.780640,212.471542,163.586258,...,102.194160,260.600006,250.334152,32.278294,117.347450,392.299591,316.019989,229.379425,166.187210,57.537430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,141.419998,104.709999,163.699997,99.200691,96.510002,286.609985,133.270004,214.460007,191.050003,158.440002,...,98.480003,96.230003,282.690002,36.119999,93.510002,524.340027,311.140015,205.199997,191.720367,96.500000
2022-09-12,142.240005,105.180000,163.490005,100.773888,96.300003,289.239990,136.449997,214.220001,192.050003,162.449997,...,99.760002,97.650002,282.829987,36.259998,94.980003,531.250000,313.399994,206.630005,192.509048,97.610001
2022-09-13,138.529999,102.199997,158.490005,95.476807,90.389999,277.720001,126.820000,207.850006,184.669998,155.350006,...,95.790001,95.010002,279.480011,35.930000,89.860001,513.960022,294.670013,199.669998,185.940002,95.330002


In [ ]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004423,0.011185,-0.010431,0.004739,-0.016102,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013996,-0.002891,-0.018919,0.004302,-0.014527,0.001645,-0.000592,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005979,-0.006292,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011751,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,0.006405,0.007021,-0.004379,0.015079,0.029001,0.015124,0.026575,0.012416,0.010579,0.032048,...,0.015781,0.000624,-0.009946,-0.003036,0.013549,-0.002890,0.020365,0.022371,0.004919,0.016753
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358


In [ ]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007921,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012986,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002854
2021-06-02,0.007964,-0.000704,-0.002232,-0.001213,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000469,0.001981,-0.010568,-0.009988,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028993,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028495
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003412,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017855,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032371,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,39,22,16,18,1,26,7,29,9,38,...,24,2,42,36,23,33,21,17,35,13
2021-06-02,35,14,6,12,19,16,45,23,8,37,...,9,25,1,18,39,40,3,42,24,10
2021-06-03,37,28,32,15,16,12,45,43,9,39,...,4,41,10,42,33,34,6,36,30,5
2021-06-04,28,18,9,33,5,22,20,30,38,27,...,13,39,32,43,3,16,17,11,45,4
2021-06-07,35,34,12,9,10,3,44,43,22,6,...,1,27,2,13,8,32,5,16,30,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,14,16,2,30,40,31,39,26,23,41,...,33,6,1,3,28,4,35,37,11,34
2022-08-10,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-08-11,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37


In [ ]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,14,16,2,30,40,31,39,26,23,41,...,33,6,1,3,28,4,35,37,11,34
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37


In [ ]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-03,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-04,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-07,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-08,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-09-12,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-09-13,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
rd3.loc[rd3.index=='2022-08-19']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-19,4,4,1,1,1,2,1,3,4,2,...,2,1,2,5,1,5,2,3,4,5


#####Functions
***

In [ ]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [ ]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-79] #-44
      
      Y_train= y.iloc[:-79] #-44

      print(Y_train.tail(2))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(58,0,-1): #from 4/01 and later -->predictions 23 
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [ ]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [ ]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 29 May-24 June** 

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-06-25') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-05-27') & (stock['ABBV'].index <= '2022-06-24')]))
# stock['ABBV']

18


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-03']['Rank'])

Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    5.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.2947078  0.19584597 0.1310139  0.28165524 0.09677709]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.08385516 0.14157292 0.18807885 0.14533    0.44116307]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16106184 0.6299727  0.05043619 0.13284531 0.02568396]
Date
2022-04-20    1.0
2022-04-21    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.29980583 0.33658979 0.01067019 0.12299822 0.22993596]
Date
2022-04-20    4.0
2022-04-21    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeI

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.65467497 0.07943653 0.08638195 0.04747726 0.1320293 ]
Date
2022-04-20    4.0
2022-04-21    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.2724747  0.10781673 0.20977816 0.28499756 0.12493286]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeI

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.074986,0.224778,0.277843,0.085833,0.144704,0.174106,0.136903,0.217757,0.097523,0.190640,...,0.130277,0.119746,0.173575,0.290503,0.233915,0.133790,0.080548,0.061546,0.129323,0.371395
2022-05-23,0.125763,0.235775,0.141738,0.086909,0.145593,0.105229,0.081878,0.467233,0.296115,0.120103,...,0.187608,0.368613,0.105708,0.084298,0.073456,0.293803,0.471835,0.137542,0.472642,0.273035
2022-05-24,0.121024,0.189443,0.059199,0.111301,0.085698,0.162898,0.331139,0.223959,0.137377,0.196039,...,0.208413,0.091256,0.040201,0.056996,0.122880,0.211332,0.386563,0.244469,0.088833,0.079173
2022-05-25,0.196191,0.228965,0.294296,0.135141,0.150072,0.086761,0.279949,0.064577,0.089379,0.127201,...,0.140466,0.082892,0.128665,0.109239,0.034004,0.209476,0.162152,0.099094,0.465874,0.084440
2022-05-26,0.205834,0.078029,0.090564,0.107627,0.256259,0.345615,0.118841,0.060604,0.165732,0.275146,...,0.172923,0.147226,0.141478,0.269495,0.085079,0.267171,0.226315,0.121212,0.056756,0.430538
2022-05-27,0.068877,0.168686,0.245693,0.074921,0.135607,0.245012,0.394454,0.060331,0.079037,0.089035,...,0.065266,0.098197,0.251525,0.128348,0.111047,0.067865,0.150288,0.145719,0.060511,0.434960
2022-05-31,0.107266,0.064521,0.070407,0.169498,0.250939,0.425488,0.129268,0.109709,0.070327,0.116963,...,0.089990,0.314754,0.128014,0.254356,0.328161,0.116438,0.227401,0.102280,0.135215,0.105584
2022-06-01,0.153980,0.289945,0.149811,0.124702,0.110535,0.183632,0.149239,0.126920,0.070644,0.092855,...,0.286899,0.332843,0.249513,0.041354,0.391054,0.249420,0.112199,0.109243,0.098104,0.347840
2022-06-02,0.231057,0.274037,0.073704,0.068842,0.092271,0.173364,0.095196,0.115684,0.276600,0.297716,...,0.116856,0.089993,0.142232,0.250126,0.115720,0.215571,0.422051,0.105783,0.088478,0.349171


In [ ]:
print(RPS1.mean(axis=1).mean())

0.17052222626953983


In [ ]:
print(RPS1.mean(axis=1))

date
2022-05-20    0.162387
2022-05-23    0.199758
2022-05-24    0.143239
2022-05-25    0.144025
2022-05-26    0.161705
2022-05-27    0.155506
2022-05-31    0.173039
2022-06-01    0.173427
2022-06-02    0.169554
2022-06-03    0.163888
2022-06-06    0.161018
2022-06-07    0.161206
2022-06-08    0.174545
2022-06-09    0.166011
2022-06-10    0.179012
2022-06-13    0.186237
2022-06-14    0.192543
2022-06-15    0.187707
2022-06-16    0.203582
2022-06-17    0.196274
2022-06-21    0.157718
2022-06-22    0.155553
2022-06-23    0.147942
2022-06-24    0.176659
dtype: float64


In [ ]:
#save results 
RPS1.to_excel('29_May_24_June-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('29_May_24_June-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances 
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15991028 0.16409869 0.1884272  0.22628162 0.26128221]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17370807 0.22414827 0.21328264 0.18992946 0.19893157]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.077955,0.227795,0.285877,0.070658,0.142149,0.143218,0.292946,0.253555,0.105347,0.247831,...,0.113862,0.133711,0.133730,0.224792,0.132950,0.096890,0.202095,0.079892,0.289340,0.282743
2022-05-23,0.147214,0.262078,0.124114,0.070894,0.140291,0.094646,0.160248,0.248161,0.275980,0.073076,...,0.262975,0.303953,0.112109,0.082247,0.128303,0.289458,0.236351,0.127477,0.259407,0.162554
2022-05-24,0.095640,0.226332,0.058245,0.118967,0.117756,0.145362,0.182723,0.253913,0.128619,0.246590,...,0.115472,0.144542,0.073544,0.082224,0.212455,0.216310,0.238471,0.225922,0.124371,0.177894
2022-05-25,0.094853,0.226532,0.247023,0.118717,0.143314,0.074993,0.181514,0.116053,0.100789,0.074147,...,0.259780,0.145435,0.112845,0.133496,0.214359,0.220339,0.106060,0.111874,0.259194,0.179267
2022-05-26,0.095124,0.104318,0.124347,0.120923,0.220709,0.283684,0.287878,0.072622,0.131615,0.241285,...,0.116642,0.129788,0.131822,0.224978,0.110846,0.223777,0.204261,0.127656,0.284213,0.281649
2022-05-27,0.080516,0.227704,0.246010,0.070870,0.138883,0.147214,0.188644,0.073199,0.100572,0.130636,...,0.069152,0.139899,0.215786,0.111564,0.210527,0.066289,0.233175,0.111016,0.277618,0.277047
2022-05-31,0.150109,0.104876,0.058022,0.119519,0.218289,0.284200,0.271968,0.116168,0.066555,0.075308,...,0.118680,0.301704,0.073603,0.249404,0.134461,0.130223,0.126711,0.080134,0.053904,0.181391
2022-06-01,0.094557,0.264199,0.124882,0.117486,0.118268,0.145347,0.272884,0.116776,0.066997,0.075398,...,0.250001,0.303292,0.273385,0.080353,0.255134,0.270592,0.140174,0.126504,0.120717,0.272973
2022-06-02,0.290710,0.225378,0.102866,0.070445,0.140343,0.146721,0.156027,0.117602,0.270732,0.235451,...,0.122465,0.142165,0.213193,0.225430,0.112563,0.228097,0.208575,0.079884,0.266335,0.270307


In [ ]:
print(RPS2.mean(axis=1).mean())

0.1594662607212629


In [ ]:
print(RPS2.mean(axis=1))

date
2022-05-20    0.165398
2022-05-23    0.165805
2022-05-24    0.150601
2022-05-25    0.151251
2022-05-26    0.161279
2022-05-27    0.155479
2022-05-31    0.161529
2022-06-01    0.169530
2022-06-02    0.155423
2022-06-03    0.161632
2022-06-06    0.153873
2022-06-07    0.153029
2022-06-08    0.159658
2022-06-09    0.161694
2022-06-10    0.161179
2022-06-13    0.164105
2022-06-14    0.163230
2022-06-15    0.163917
2022-06-16    0.163018
2022-06-17    0.167833
2022-06-21    0.151013
2022-06-22    0.152368
2022-06-23    0.152378
2022-06-24    0.161967
dtype: float64


In [ ]:

#save results 
RPS2.to_excel('29_May_24_June-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('29_May_24_June-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

####**For 26 June.-22 July.**

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-06-01','2022-07-23') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-06-01  113.699997  113.989998  112.040001  112.209999  106.605812
 2021-06-02  112.410004  112.660004  110.849998  111.400002  105.836273
 2021-06-03  111.120003  112.410004  110.650002  112.209999  106.605812
 2021-06-04  112.400002  113.110001  112.160004  112.360001  106.748337
 2021-06-07  112.709999  114.110001  112.330002  113.010002  107.365868
 ...                ...         ...         ...         ...         ...
 2022-07-18  153.800003  154.470001  148.889999  149.570007  149.570007
 2022-07-19  150.000000  150.229996  148.800003  149.740005  149.740005
 2022-07-20  149.529999  150.449997  147.339996  147.690002  147.690002
 2022-07-21  147.240005  147.839996  146.020004  147.750000  147.750000
 2022-07-22  148.470001  149.289993  147.580002  148.470001  148.470001
 
 [289 rows x 5 columns],
 'AEP':                  Open

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-06-26') & (stock['ABBV'].index <= '2022-07-22')]))

19


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-26']['Rank'])

Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, d

#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-05-17    2
2022-05-18    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19191561 0.11804327 0.28696161 0.15512509 0.24795442]
Date
2022-05-17    1
2022-05-18    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16793271 0.17921596 0.21465767 0.26207419 0.17611948]
Date
2022-05-17    4
2022-05-18    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18449868 0.27621358 0.21156656 0.29480885 0.03291233]
Date
2022-05-17    4
2022-05-18    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19243526 0.16671573 0.38469584 0.13093012 0.12522304]
Date
2022-05-17    3
2022-05-18    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.2755696  0.18333888 0.17713147 0.11879442 0.24516563]
Date
2022-05-17    5
2022-05-18    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19937877 0.14104263 0.22979357 0.27469456 0.15509047]
Date
2022-05-17    4
2022-05-18    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19869

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.196070,0.088121,0.218558,0.508392,0.261813,0.328784,0.408899,0.055082,0.358514,0.100702,...,0.064601,0.267027,0.150416,0.126737,0.158138,0.099689,0.157933,0.095491,0.191503,0.061843
2022-06-21,0.148227,0.262968,0.044988,0.174219,0.170019,0.114193,0.040629,0.188597,0.250581,0.257154,...,0.082251,0.132381,0.282627,0.128347,0.075585,0.096731,0.205899,0.131997,0.050183,0.123550
2022-06-22,0.139499,0.255632,0.156483,0.003543,0.025673,0.061716,0.056263,0.240134,0.116909,0.136288,...,0.334373,0.180980,0.098822,0.121720,0.187821,0.338018,0.148165,0.073053,0.289408,0.469211
2022-06-23,0.265814,0.236032,0.100280,0.163629,0.075510,0.053713,0.069330,0.086528,0.109120,0.216106,...,0.096073,0.121060,0.061105,0.107594,0.215118,0.181179,0.145416,0.091625,0.111461,0.186166
2022-06-24,0.161981,0.292022,0.129607,0.169412,0.336292,0.475595,0.453432,0.085016,0.453974,0.122753,...,0.039769,0.302750,0.261374,0.227577,0.164493,0.063384,0.577453,0.208107,0.166605,0.170701
2022-06-27,0.303506,0.108317,0.100257,0.003385,0.230817,0.142128,0.097301,0.207998,0.364936,0.099284,...,0.052444,0.146924,0.173440,0.249270,0.089020,0.077649,0.525367,0.058206,0.087243,0.116358
2022-06-28,0.301759,0.101175,0.139575,0.247725,0.152084,0.273594,0.351234,0.061312,0.105748,0.257891,...,0.119307,0.110005,0.129661,0.193913,0.161528,0.086781,0.536768,0.085387,0.050939,0.216739
2022-06-29,0.282014,0.208561,0.047315,0.056648,0.207469,0.255131,0.139165,0.189211,0.061661,0.096388,...,0.060349,0.346244,0.289637,0.314142,0.255714,0.240629,0.102236,0.060094,0.126325,0.371284
2022-06-30,0.124913,0.086094,0.263746,0.258037,0.122865,0.213041,0.316048,0.156795,0.090170,0.091602,...,0.289724,0.385158,0.204906,0.080015,0.313557,0.166383,0.062110,0.310652,0.143094,0.095175


In [ ]:
print(RPS1.mean(axis=1).mean())

0.1677141491205599


In [ ]:
print(RPS1.mean(axis=1))

date
2022-06-17    0.196663
2022-06-21    0.152879
2022-06-22    0.170039
2022-06-23    0.147708
2022-06-24    0.204124
2022-06-27    0.170997
2022-06-28    0.194275
2022-06-29    0.163769
2022-06-30    0.177754
2022-07-01    0.170150
2022-07-05    0.185704
2022-07-06    0.156826
2022-07-07    0.161014
2022-07-08    0.159486
2022-07-11    0.162000
2022-07-12    0.148225
2022-07-13    0.157551
2022-07-14    0.182421
2022-07-15    0.162930
2022-07-18    0.151350
2022-07-19    0.164171
2022-07-20    0.151065
2022-07-21    0.161703
2022-07-22    0.172336
dtype: float64


In [ ]:
#save results 
RPS1.to_excel('26_June_22_July-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('26_June_22_July-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-05-17    2
2022-05-18    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1414291  0.19781497 0.19437629 0.21238043 0.2539992 ]
Date
2022-05-17    1
2022-05-18    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18272005 0.23617089 0.15805824 0.22841337 0.19463745]
Date
2022-05-17    4
2022-05-18    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.291431,0.112483,0.122231,0.274265,0.202456,0.108252,0.155331,0.078493,0.134678,0.075785,...,0.108021,0.286026,0.129139,0.091867,0.126930,0.071278,0.144279,0.117183,0.124242,0.267497
2022-06-21,0.094218,0.247115,0.107791,0.076240,0.228799,0.068625,0.195186,0.237008,0.064659,0.237219,...,0.108417,0.142611,0.221036,0.093739,0.084632,0.071412,0.194914,0.121666,0.059790,0.162512
2022-06-22,0.094484,0.247275,0.246905,0.101868,0.140089,0.068559,0.197989,0.123576,0.093908,0.125943,...,0.273812,0.207601,0.115019,0.093209,0.223542,0.215214,0.146712,0.074678,0.244931,0.179455
2022-06-23,0.185876,0.246892,0.107737,0.076194,0.140617,0.130071,0.198528,0.114547,0.092403,0.246162,...,0.108742,0.141665,0.069821,0.125711,0.224914,0.099793,0.106357,0.120121,0.123207,0.105818
2022-06-24,0.155747,0.241301,0.121679,0.075742,0.207430,0.235311,0.242447,0.114514,0.306841,0.076866,...,0.060359,0.293813,0.271388,0.239107,0.127565,0.139004,0.249207,0.241643,0.113434,0.106539
2022-06-27,0.292488,0.079190,0.121763,0.101437,0.143392,0.103111,0.104126,0.244867,0.300984,0.114612,...,0.061778,0.130705,0.270648,0.227333,0.084224,0.068440,0.251198,0.073831,0.122907,0.107441
2022-06-28,0.292478,0.080572,0.108181,0.131860,0.121363,0.229057,0.245918,0.077020,0.096911,0.128194,...,0.124368,0.138974,0.070709,0.239363,0.224534,0.137738,0.247723,0.117229,0.062039,0.182745
2022-06-29,0.291976,0.241718,0.121854,0.076202,0.146059,0.273238,0.153666,0.243362,0.064711,0.110818,...,0.064486,0.296911,0.271732,0.227769,0.248809,0.286323,0.122736,0.072863,0.112557,0.272262
2022-06-30,0.094719,0.081571,0.281762,0.219090,0.137896,0.136561,0.125370,0.243604,0.129401,0.110990,...,0.243229,0.293655,0.128710,0.128339,0.226499,0.098671,0.198186,0.244343,0.123796,0.104609


In [ ]:
print(RPS2.mean(axis=1).mean())

0.1586875236700397


In [ ]:
print(RPS2.mean(axis=1))

date
2022-06-17    0.168727
2022-06-21    0.149526
2022-06-22    0.153109
2022-06-23    0.151084
2022-06-24    0.164860
2022-06-27    0.162327
2022-06-28    0.162266
2022-06-29    0.165010
2022-06-30    0.163592
2022-07-01    0.165111
2022-07-05    0.162836
2022-07-06    0.150303
2022-07-07    0.157630
2022-07-08    0.148380
2022-07-11    0.166463
2022-07-12    0.162484
2022-07-13    0.167262
2022-07-14    0.168818
2022-07-15    0.162462
2022-07-18    0.147262
2022-07-19    0.157929
2022-07-20    0.147044
2022-07-21    0.146838
2022-07-22    0.157175
dtype: float64


In [ ]:
#save results 
RPS2.to_excel('26_June_22_July-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('26_June_22_July-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

####**For 24 July-19 August**

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-07-01','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-07-01  112.949997  114.550003  112.889999  114.260002  108.553429
 2021-07-02  114.519997  115.489998  113.889999  115.169998  109.417984
 2021-07-06  115.169998  115.900002  114.769997  115.730003  109.950020
 2021-07-07  115.910004  116.970001  115.309998  116.750000  110.919067
 2021-07-08  115.760002  116.809998  115.680000  116.349998  110.539055
 ...                ...         ...         ...         ...         ...
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993
 2022-08-16  142.110001  143.979996  142.080002  142.550003  142.550003
 2022-08-17  141.419998  142.759995  140.550003  141.440002  141.440002
 2022-08-18  141.490005  141.580002  139.820007  141.289993  141.289993
 2022-08-19  141.779999  143.360001  141.460007  141.850006  141.850006
 
 [287 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-07-24') & (stock['ABBV'].index <= '2022-08-19')]))

20


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-06-23']['Rank'])

Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    1.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    1.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-06-09    3.0
2022-06-10    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15907855 0.12828275 0.19757523 0.16357497 0.35148851]
Date
2022-06-09    3.0
2022-06-10    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.07865905 0.32176213 0.08454884 0.15621555 0.35881443]
Date
2022-06-09    5.0
2022-06-10    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.25236693 0.33269505 0.14935937 0.2420303  0.02354835]
Date
2022-06-09    5.0
2022-06-10    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.22241292 0.1724908  0.11084843 0.04368856 0.45055929]
Date
2022-06-09    1.0
2022-06-10    2.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeI

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.29685053 0.21454932 0.10820816 0.15813489 0.2222571 ]
Date
2022-06-09    3.0
2022-06-10    2.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.33993656 0.17392996 0.20225397 0.18481928 0.09906023]
Date
2022-06-09    1.0
2022-06-10    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeI

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,0.295026,0.217768,0.040634,0.245399,0.217505,0.095266,0.189783,0.103042,0.061261,0.085893,...,0.112256,0.127972,0.138213,0.300255,0.097566,0.133723,0.152451,0.221249,0.212926,0.316560
2022-07-14,0.141828,0.101914,0.093605,0.092952,0.206196,0.243044,0.171929,0.058355,0.254883,0.129129,...,0.249136,0.222785,0.121546,0.154310,0.189966,0.065008,0.139121,0.275444,0.397437,0.283285
2022-07-15,0.140080,0.123309,0.211513,0.275442,0.178668,0.155052,0.163168,0.054746,0.273449,0.103617,...,0.071453,0.248476,0.093624,0.203400,0.182268,0.060599,0.129755,0.074670,0.145271,0.348330
2022-07-18,0.064808,0.072076,0.184186,0.103465,0.156788,0.117377,0.229888,0.201599,0.098287,0.114895,...,0.070655,0.183806,0.241880,0.158908,0.084562,0.136146,0.304620,0.085733,0.078554,0.139635
2022-07-19,0.162128,0.167958,0.189340,0.093742,0.245769,0.220979,0.123687,0.307757,0.246849,0.115867,...,0.130375,0.169390,0.303699,0.085711,0.425686,0.088745,0.253667,0.077536,0.106073,0.153841
2022-07-20,0.203915,0.182375,0.088776,0.200348,0.182850,0.130674,0.260753,0.086441,0.056766,0.140518,...,0.109123,0.206163,0.119009,0.150683,0.134230,0.326469,0.152091,0.076335,0.088108,0.074724
2022-07-21,0.096263,0.084112,0.266896,0.210907,0.194654,0.140816,0.250965,0.074256,0.134373,0.156182,...,0.100875,0.206055,0.143129,0.200094,0.414549,0.304308,0.188788,0.081996,0.161385,0.113244
2022-07-22,0.155898,0.076574,0.090164,0.084069,0.292352,0.073979,0.265558,0.235545,0.053636,0.099653,...,0.266831,0.108287,0.097604,0.278167,0.085020,0.237391,0.086391,0.098650,0.249948,0.095437
2022-07-25,0.259243,0.123725,0.107499,0.072142,0.279924,0.081484,0.079193,0.327400,0.114702,0.244629,...,0.058980,0.203350,0.124760,0.285217,0.131743,0.077124,0.225324,0.110251,0.111037,0.083421


In [ ]:
print(RPS1.mean(axis=1).mean())

0.166204956004358


In [ ]:
print(RPS1.mean(axis=1))

date
2022-07-13    0.163885
2022-07-14    0.183470
2022-07-15    0.165945
2022-07-18    0.152379
2022-07-19    0.168622
2022-07-20    0.148366
2022-07-21    0.160279
2022-07-22    0.155028
2022-07-25    0.168971
2022-07-26    0.173937
2022-07-27    0.153123
2022-07-28    0.152675
2022-07-29    0.168728
2022-08-01    0.164662
2022-08-02    0.171600
2022-08-03    0.154761
2022-08-04    0.174713
2022-08-05    0.169311
2022-08-08    0.168102
2022-08-09    0.186713
2022-08-10    0.170891
2022-08-11    0.154010
2022-08-12    0.166326
2022-08-15    0.192423
dtype: float64


In [ ]:
#save results 
#save results 
RPS1.to_excel('24_July_19_August-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('24_July_19_August-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-06-09    3.0
2022-06-10    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13353821 0.21001621 0.19326862 0.20067866 0.2624983 ]
Date
2022-06-09    3.0
2022-06-10    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16144842 0.22320535 0.17511327 0.22678382 0.21344913]
Date
2022-06-09    5.0
2022-06-10    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-14  142.600006  103.779999  165.597916  105.951485  110.098442   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-14  289.170013  143.550003  218.160004  201.651291  165.839996  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-14  104.265739  101.099998  273.894653  36.970001  89.804367   

                   UNH         URI           V        VRSK   XOM  
Date                                                              
2022-07-14  541.999329  336.549988  211.330002  202.532333  94.0  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,0.293220,0.210444,0.057893,0.281353,0.219207,0.111982,0.153413,0.115190,0.067190,0.074325,...,0.122726,0.136856,0.128014,0.248259,0.122239,0.107986,0.123517,0.244618,0.240799,0.262012
2022-07-14,0.143372,0.098046,0.102397,0.128675,0.210808,0.249815,0.123592,0.073343,0.239543,0.102331,...,0.243009,0.206532,0.128644,0.132890,0.128130,0.070695,0.109386,0.248273,0.267325,0.266846
2022-07-15,0.143711,0.137125,0.238684,0.280922,0.206610,0.116038,0.152048,0.073431,0.270218,0.102700,...,0.070240,0.209272,0.111013,0.224304,0.233387,0.070992,0.109398,0.070715,0.238808,0.262674
2022-07-18,0.081574,0.098216,0.238579,0.129874,0.206225,0.112803,0.200140,0.262525,0.109707,0.103585,...,0.070276,0.128936,0.214719,0.134435,0.124396,0.110258,0.204371,0.114295,0.062265,0.180579
2022-07-19,0.143142,0.138575,0.126469,0.128959,0.222194,0.246622,0.107261,0.234160,0.240639,0.136853,...,0.123893,0.214372,0.212802,0.079035,0.220687,0.133855,0.203317,0.071238,0.062932,0.177626
2022-07-20,0.197060,0.208628,0.104788,0.220435,0.208665,0.125038,0.196957,0.073529,0.067020,0.103136,...,0.117109,0.129779,0.108611,0.136048,0.095060,0.215638,0.123170,0.071702,0.111531,0.106874
2022-07-21,0.099757,0.098800,0.279070,0.220749,0.208955,0.125210,0.198432,0.073428,0.122915,0.103415,...,0.117578,0.129100,0.136762,0.222445,0.215402,0.218675,0.123126,0.071983,0.127533,0.176605
2022-07-22,0.145018,0.085950,0.063435,0.104725,0.218907,0.108827,0.245073,0.235826,0.067427,0.135857,...,0.241415,0.142242,0.108867,0.259063,0.131626,0.271840,0.141791,0.118459,0.238074,0.176642
2022-07-25,0.293245,0.136809,0.105125,0.104432,0.216697,0.069675,0.106439,0.255692,0.103703,0.260732,...,0.072565,0.209531,0.109578,0.257855,0.253787,0.069345,0.108921,0.119395,0.112756,0.105914


In [ ]:
print(RPS2.mean(axis=1).mean())

0.1588277187922795


In [ ]:
print(RPS2.mean(axis=1))

date
2022-07-13    0.166387
2022-07-14    0.166966
2022-07-15    0.162943
2022-07-18    0.145765
2022-07-19    0.159595
2022-07-20    0.145030
2022-07-21    0.145658
2022-07-22    0.163250
2022-07-25    0.164492
2022-07-26    0.168712
2022-07-27    0.147591
2022-07-28    0.156805
2022-07-29    0.160648
2022-08-01    0.165521
2022-08-02    0.149899
2022-08-03    0.158073
2022-08-04    0.158621
2022-08-05    0.165910
2022-08-08    0.161641
2022-08-09    0.162908
2022-08-10    0.164300
2022-08-11    0.144712
2022-08-12    0.160315
2022-08-15    0.166120
dtype: float64


In [ ]:
#save results 
RPS2.to_excel('24_July_19_August-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('24_July_19_August-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')

####For the whole period with one month less training 
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Logistic Regression(baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.23040706 0.07782861 0.22061416 0.23853378 0.23261639]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11982662 0.32844767 0.12053985 0.15575968 0.27542618]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.21867    0.27334119 0.17539774 0.08351422 0.24907685]
Date
2022-04-20    3.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.26292571 0.29680713 0.17673679 0.22453363 0.03899674]
Date
2022-04-20    1.0
2022-04-21    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.24064954 0.16425773 0.09494739 0.09094044 0.4092049 ]
Date
2022-04-20    4.0
2022-04-21    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.078644,0.227462,0.276457,0.086991,0.142645,0.169342,0.136903,0.216246,0.098913,0.195212,...,0.131579,0.119746,0.171676,0.291446,0.236557,0.131430,0.080548,0.060487,0.132641,0.367949
2022-05-23,0.126004,0.230507,0.141060,0.087405,0.150816,0.100748,0.081878,0.473993,0.294064,0.117681,...,0.186395,0.368613,0.106591,0.087094,0.063633,0.297912,0.471835,0.135182,0.469301,0.269368
2022-05-24,0.124086,0.189724,0.060291,0.115028,0.085797,0.167100,0.331139,0.222307,0.136152,0.201814,...,0.207658,0.091256,0.040451,0.057827,0.120238,0.206021,0.386563,0.248862,0.088901,0.080188
2022-05-25,0.189962,0.233075,0.293752,0.135952,0.146905,0.087510,0.279949,0.062081,0.090692,0.124964,...,0.140828,0.082892,0.127537,0.107146,0.034660,0.203629,0.162152,0.098945,0.466240,0.085146
2022-05-26,0.196433,0.078296,0.088734,0.106967,0.254053,0.343434,0.118841,0.060775,0.164667,0.281172,...,0.169953,0.147226,0.143885,0.271971,0.086729,0.266991,0.226315,0.121247,0.055260,0.429973
2022-05-27,0.074976,0.168735,0.248446,0.072690,0.140969,0.240130,0.394454,0.060092,0.079037,0.087285,...,0.063682,0.098197,0.240286,0.129674,0.083930,0.069498,0.150288,0.144876,0.057895,0.427942
2022-05-31,0.104671,0.064627,0.071944,0.169563,0.261827,0.414058,0.129268,0.110830,0.070931,0.118097,...,0.090832,0.314754,0.124145,0.250864,0.364202,0.116262,0.227401,0.101242,0.142635,0.108507
2022-06-01,0.154518,0.287930,0.149875,0.122361,0.111478,0.176092,0.149239,0.128306,0.071157,0.093890,...,0.280829,0.332843,0.257144,0.040259,0.435318,0.242341,0.112199,0.109346,0.098134,0.341340
2022-06-02,0.229159,0.280757,0.073041,0.071727,0.088687,0.163198,0.095196,0.116306,0.277536,0.302131,...,0.113222,0.089993,0.142548,0.249935,0.111411,0.218497,0.422051,0.103351,0.086795,0.350488


In [ ]:
print(RPS1.mean(axis=1).mean())

0.1685024020200905


In [ ]:
#for 24 June
print(RPS1.iloc[1].mean())
#for 22 July
print(RPS1.iloc[20].mean())
#for 19 August
print(RPS1.iloc[40].mean())

0.20040556049257854
0.1604078850953036
0.15393564543551402


In [ ]:
RPS1.mean(axis=1)

date
2022-05-20    0.162082
2022-05-23    0.200406
2022-05-24    0.143676
2022-05-25    0.143257
2022-05-26    0.160535
2022-05-27    0.154257
2022-05-31    0.174442
2022-06-01    0.173866
2022-06-02    0.168711
2022-06-03    0.163709
2022-06-06    0.160854
2022-06-07    0.161691
2022-06-08    0.174512
2022-06-09    0.166030
2022-06-10    0.179516
2022-06-13    0.186336
2022-06-14    0.193189
2022-06-15    0.188917
2022-06-16    0.202782
2022-06-17    0.198350
2022-06-21    0.160408
2022-06-22    0.154010
2022-06-23    0.150135
2022-06-24    0.209611
2022-06-27    0.175426
2022-06-28    0.191564
2022-06-29    0.174836
2022-06-30    0.175741
2022-07-01    0.178534
2022-07-05    0.186418
2022-07-06    0.157186
2022-07-07    0.162638
2022-07-08    0.154970
2022-07-11    0.166682
2022-07-12    0.148535
2022-07-13    0.156811
2022-07-14    0.184712
2022-07-15    0.166730
2022-07-18    0.155004
2022-07-19    0.162155
2022-07-20    0.153936
2022-07-21    0.162670
2022-07-22    0.149918
2022-0

#####After feature importances
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15128556 0.17883277 0.21301754 0.22267837 0.23418576]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16587575 0.22541227 0.18386335 0.22885142 0.19599722]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.077955,0.227795,0.285877,0.070658,0.142149,0.143218,0.292946,0.253555,0.105347,0.247831,...,0.113862,0.133711,0.133730,0.224792,0.132950,0.096890,0.202095,0.079892,0.289340,0.282743
2022-05-23,0.147214,0.262078,0.124114,0.070894,0.140291,0.094646,0.160248,0.248161,0.275980,0.073076,...,0.262975,0.303953,0.112109,0.082247,0.128303,0.289458,0.236351,0.127477,0.259407,0.162554
2022-05-24,0.095640,0.226332,0.058245,0.118967,0.117756,0.145362,0.182723,0.253913,0.128619,0.246590,...,0.115472,0.144542,0.073544,0.082224,0.212455,0.216310,0.238471,0.225922,0.124371,0.177894
2022-05-25,0.094853,0.226532,0.247023,0.118717,0.143314,0.074993,0.181514,0.116053,0.100789,0.074147,...,0.259780,0.145435,0.112845,0.133496,0.214359,0.220339,0.106060,0.111874,0.259194,0.179267
2022-05-26,0.095124,0.104318,0.124347,0.120923,0.220709,0.283684,0.287878,0.072622,0.131615,0.241285,...,0.116642,0.129788,0.131822,0.224978,0.110846,0.223777,0.204261,0.127656,0.284213,0.281649
2022-05-27,0.080516,0.227704,0.246010,0.070870,0.138883,0.147214,0.188644,0.073199,0.100572,0.130636,...,0.069152,0.139899,0.215786,0.111564,0.210527,0.066289,0.233175,0.111016,0.277618,0.277047
2022-05-31,0.150109,0.104876,0.058022,0.119519,0.218289,0.284200,0.271968,0.116168,0.066555,0.075308,...,0.118680,0.301704,0.073603,0.249404,0.134461,0.130223,0.126711,0.080134,0.053904,0.181391
2022-06-01,0.094557,0.264199,0.124882,0.117486,0.118268,0.145347,0.272884,0.116776,0.066997,0.075398,...,0.250001,0.303292,0.273385,0.080353,0.255134,0.270592,0.140174,0.126504,0.120717,0.272973
2022-06-02,0.290710,0.225378,0.102866,0.070445,0.140343,0.146721,0.156027,0.117602,0.270732,0.235451,...,0.122465,0.142165,0.213193,0.225430,0.112563,0.228097,0.208575,0.079884,0.266335,0.270307


In [ ]:
print(RPS2.mean(axis=1).mean())

0.15896453591605755


In [ ]:
RPS2.mean(axis=1)


date
2022-05-20    0.165398
2022-05-23    0.165805
2022-05-24    0.150601
2022-05-25    0.151251
2022-05-26    0.161279
2022-05-27    0.155479
2022-05-31    0.161529
2022-06-01    0.169530
2022-06-02    0.155423
2022-06-03    0.161632
2022-06-06    0.153873
2022-06-07    0.153029
2022-06-08    0.159658
2022-06-09    0.161694
2022-06-10    0.161179
2022-06-13    0.164105
2022-06-14    0.163230
2022-06-15    0.163917
2022-06-16    0.163018
2022-06-17    0.167833
2022-06-21    0.151013
2022-06-22    0.152368
2022-06-23    0.152378
2022-06-24    0.164528
2022-06-27    0.162156
2022-06-28    0.161286
2022-06-29    0.165425
2022-06-30    0.162818
2022-07-01    0.164083
2022-07-05    0.162274
2022-07-06    0.151094
2022-07-07    0.156892
2022-07-08    0.149283
2022-07-11    0.163215
2022-07-12    0.158703
2022-07-13    0.164808
2022-07-14    0.167176
2022-07-15    0.160919
2022-07-18    0.149543
2022-07-19    0.156857
2022-07-20    0.149590
2022-07-21    0.149114
2022-07-22    0.158305
2022-0

In [ ]:
#for 24 June
print(RPS2.iloc[1].mean())
#for 22 July
print(RPS2.iloc[20].mean())
#for 19 August
print(RPS2.iloc[40].mean())

0.16580516236896573
0.1510132733344073
0.14958952521542782


In [ ]:
#SAVE FINAL RESULTS
RPS2.to_excel('Q2-submissions-running-for-the-whole-period.xlsx')